In [4]:
pip install pygame


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\kjaer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -- ------------------------------------- 0.6/10.6 MB 13.3 MB/s eta 0:00:01
   ----- ---------------------------------- 1.5/10.6 MB 15.7 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 16.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/10.6 MB 16.7 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.6 MB 17.2 MB/s eta 0:00:01
   ----------------- ---------------------- 4.6/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/10.6 MB 16.7 MB/s eta 0:00:01
   ----------------------- ---------------- 6.2/10.6 MB 16.6 MB/s eta 0:00:01
   -------------------------- ------------- 7.0/10.6 MB 16.5 MB/s eta 0:00:01
   ---------------------------- ----------- 7.7/10.6 MB 16.4 MB/s eta 0:00:01
   -------------------------------- ------- 8.5/10.6 MB 16.1 MB/s eta 0:00:01
   ---------------------------------- ----- 9.2/10.6 MB 16.4 MB/s eta 0

In [ ]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
width, height = 840, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("El Examen")

# Colors
white = (255, 255, 255)
black = (0, 0, 0)

#Font
font = pygame.font.Font(None,32)
font_2 = pygame.font.Font(None,25)

#Buttons
button_width, button_height = 120, 50

button_color = black
button_math = pygame.Rect(120, 150, button_width, button_height)
button_chem = pygame.Rect(360, 150, button_width, button_height)
button_eng = pygame.Rect(600, 150, button_width, button_height)

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second

running = True
i=0

def button_dynamics(button, name, subname_1, subname_2, subname_3):
        hover = button.collidepoint(mouse_x, mouse_y)

        if hover and button.height < 350:
            button.height += 10
        elif not hover and button.height > 50:
            button.height -= 15
        
        button = pygame.Rect(button.x, button.y, button.width, button.height)
        pygame.draw.rect(screen, black, button,0, 30)

        text = font.render(name, True, white)
        text_button = text.get_rect(center=(button.centerx, button.y+25))
        screen.blit(text, text_button)

        subbutton_1 = subbutton_2 = subbutton_3 = None

        if button.height > 110:
            subbutton_1 = pygame.Rect(button.x+10,button.y+60,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_1,0, 30)
            text_1 = font_2.render(subname_1, True, black)
            text_subbutton_1 = text_1.get_rect(center=subbutton_1.center)
            screen.blit(text_1, text_subbutton_1)

        if button.height > 210:
            subbutton_2 = pygame.Rect(button.x+10,button.y+160,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_2,0, 30)
            text_2 = font_2.render(subname_2, True, black)
            text_subbutton_2 = text_2.get_rect(center=subbutton_2.center)
            screen.blit(text_2, text_subbutton_2)

        if button.height > 310:
            subbutton_3 = pygame.Rect(button.x+10,button.y+260,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_3,0, 30)
            text_3 = font_2.render(subname_3, True, black)
            text_subbutton_3 = text_3.get_rect(center=subbutton_3.center)
            screen.blit(text_3, text_subbutton_3)
        
        return subbutton_1, subbutton_2, subbutton_3

def Math_Game_Uno():
    math_game_uno = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Math uno')
    math_game_uno.fill(white)
    pygame.display.flip()
    points = []

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                pygame.display.set_caption('El Examen')
                return
            elif event.type == pygame.MOUSEBUTTONDOWN:
                set_points()
        def set_points():
            for i in range(100):
                points.append((random.randint(0,100), random.randint(0,100)))
            for point in points:
                pygame.draw.circle(screen, black, (point[0]+width/2-100, point[1]+height/2-100), 2)
        pygame.display.update()

def Math_Game_Dos():
    math_game_dos = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Math dos')
    math_game_dos.fill(white)
    pygame.display.flip()
    points = []

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                pygame.display.set_caption('El Examen')
                return
            elif event.type == pygame.MOUSEBUTTONDOWN:
                set_points()
        def set_points():
            for i in range(100):
                points.append((random.randint(0,100), random.randint(0,100)))
            for point in points:
                pygame.draw.circle(screen, black, (point[0]+width/2-100, point[1]+height/2-100), 2)
        pygame.display.update()
    
def Math_Game_Tres():
    math_game_tres = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Math tres')
    math_game_tres.fill(white)
    pygame.display.flip()
    points = []

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                pygame.display.set_caption('El Examen')
                return
            elif event.type == pygame.MOUSEBUTTONDOWN:
                set_points()
        def set_points():
            for i in range(100):
                points.append((random.randint(0,200), random.randint(0,200)))
            for point in points:
                pygame.draw.circle(screen, black, (point[0]+width/2, point[1]+height/2), 3)
        pygame.display.update()

def Eng_Game_Uno():  
    button_4 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_4,0, 30)
    text_eng_1 = font_2.render("English Game 1", True, white)
    text_eng_1_1 = text_eng_1.get_rect(center=button_4.center)
    screen.blit(text_eng_1, text_eng_1_1)

def Eng_Game_Dos(): 
    button_5 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_5,0, 30)
    text_eng_2 = font_2.render("English Game 2", True, white)
    text_eng_1_2 = text_eng_2.get_rect(center=button_5.center)
    screen.blit(text_eng_2, text_eng_1_2)

def Eng_Game_Tres(): 
    button_6 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_6,0, 30)
    text_eng_3 = font_2.render("English Game 3", True, white)
    text_eng_1_3 = text_eng_3.get_rect(center=button_6.center)
    screen.blit(text_eng_3, text_eng_1_3)

def Chem_Game_Uno(): 
    button_7 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_7,0, 30)
    text_chem_1 = font_2.render("Chemistry Game 1", True, white)
    text_chem_1_1 = text_chem_1.get_rect(center=button_7.center)
    screen.blit(text_chem_1, text_chem_1_1) 

def Chem_Game_Dos(): 
    button_8 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_8,0, 30)
    text_chem_2 = font_2.render("Chemistry Game 2", True, white)
    text_chem_1_2 = text_chem_2.get_rect(center=button_8.center)
    screen.blit(text_chem_2, text_chem_1_2) 

def Chem_Game_Tres(): 
    button_9 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_9,0, 30)
    text_chem_3 = font_2.render("Chemistry Game 3", True, white)
    text_chem_1_3 = text_chem_3.get_rect(center=button_9.center)
    screen.blit(text_chem_3, text_chem_1_3) 


while running:
    screen.fill(white)

    mouse_x, mouse_y = pygame.mouse.get_pos()

    home_button = pygame.Rect(20,20,50,50)
    pygame.draw.rect(screen, black, home_button,0, 15)
    home_text = font.render("[]", True, white)
    home_text_1 = home_text.get_rect(center=home_button.center)
    screen.blit(home_text, home_text_1)

    
    subbutton_math_1, subbutton_math_2, subbutton_math_3 = button_dynamics(button_math, 'Math','Game 1','Game 2','Game 3')
    subbutton_eng_1, subbutton_eng_2, subbutton_eng_3 = button_dynamics(button_chem, 'Chemistry','Game 1','Game 2','Game 3')
    subbutton_chem_1, subbutton_chem_2, subbutton_chem_3 = button_dynamics(button_eng,'English','Game 1','Game 2','Game 3')
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
                if subbutton_math_1 and subbutton_math_1.collidepoint(mouse_x, mouse_y):
                    Math_Game_Uno()
                if subbutton_math_2 and subbutton_math_2.collidepoint(mouse_x, mouse_y):
                    Math_Game_Dos()
                if subbutton_math_3 and subbutton_math_3.collidepoint(mouse_x, mouse_y):
                    Math_Game_Tres()
                if subbutton_eng_1 and subbutton_eng_1.collidepoint(mouse_x, mouse_y):
                    Eng_Game_Uno()
                if subbutton_eng_2 and subbutton_eng_2.collidepoint(mouse_x, mouse_y):
                    Eng_Game_Dos()
                if subbutton_eng_3 and subbutton_eng_3.collidepoint(mouse_x, mouse_y):
                    Eng_Game_Tres()
                if subbutton_chem_1 and subbutton_chem_1.collidepoint(mouse_x, mouse_y):
                    Chem_Game_Uno()
                if subbutton_chem_2 and subbutton_chem_2.collidepoint(mouse_x, mouse_y):
                    Chem_Game_Dos()
                if subbutton_chem_3 and subbutton_chem_3.collidepoint(mouse_x, mouse_y):
                    Chem_Game_Tres()
                
    

# Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()

In [25]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WINDOW_SIZE = (400, 300)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)

# Set up the main window
main_window = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption('Main Window')

# Button properties
button_rect = pygame.Rect(150, 100, 100, 50)

def open_new_window():
    new_window = pygame.display.set_mode(WINDOW_SIZE)
    pygame.display.set_caption('New Window')

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                return

        new_window.fill(WHITE)
        pygame.display.update()

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if button_rect.collidepoint(event.pos):
                open_new_window()

    main_window.fill(WHITE)
    pygame.draw.rect(main_window, BLUE, button_rect)
    pygame.display.flip()

pygame.quit()


SystemExit: 

C:\Users\kjaer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
